In [27]:
import torch

In [28]:
torch.cuda.is_available()

False

In [29]:
import numpy as np
from torchvision import datasets,transforms

In [30]:
transform=transforms.ToTensor()

train_data=datasets.MNIST(root='../MNIST_data',download=True,train=True,transform=transform)
test_data=datasets.MNIST(root='../MNIST_data',download=True,train=False,transform=transform)

In [31]:
# from torch.utils.data.sampler import SubsetRandomSampler,random_split
# valid_size = 0.2

# # obtain training indices that will be used for validation
# num_train = len(train_data)
# print(num_train)
# indices = list(range(num_train))
# np.random.shuffle(indices)
# split = int(np.floor(valid_size * num_train))
# print(split)
# train_idx, valid_idx = indices[split:], indices[:split]

# # define samplers for obtaining training and validation batches
# #train_sampler = SubsetRandomSampler(train_idx)
# #valid_sampler = SubsetRandomSampler(valid_idx)

In [32]:
from torch.utils.data import SubsetRandomSampler,random_split
train_set,validation_set=random_split(train_data,[int(0.8*len(train_data)),int(0.2*len(train_data))])


In [33]:
from torch.utils.data import DataLoader
batch_size=20
train_loader=DataLoader(train_set,batch_size=batch_size)
valid_loader=DataLoader(validation_set,batch_size=batch_size)
test_loader=DataLoader(test_data,batch_size=batch_size)

In [34]:
import matplotlib.pyplot as plt
    
# obtain one batch of training images
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy()

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(batch_size):
    ax = fig.add_subplot(2, batch_size//2, idx+1, xticks=[], yticks=[])
    ax.imshow(np.squeeze(images[idx]), cmap='gray')
    # print out the correct label for each image
    # .item() gets the value contained in a Tensor
    ax.set_title(str(labels[idx].item()))

In [35]:
import torch.nn as nn
model=nn.Sequential(
                      nn.Linear(784, 128),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Dropout(0.2),
                      nn.Linear(64, 10),
                   )

print(model)

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2)
  (3): Linear(in_features=128, out_features=64, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.2)
  (6): Linear(in_features=64, out_features=10, bias=True)
)


In [36]:
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(model.parameters(),lr=0.01)

In [37]:
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
    # monitor training loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train() # prep model for training
    for data, target in train_loader:
        data=data.view(data.shape[0], -1)
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update running training loss
        train_loss += loss.item()*data.size(0)
        
    ######################    
    # validate the model #
    ######################
    model.eval() # prep model for evaluation
    for data, target in valid_loader:
        data=data.view(data.shape[0], -1)
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the loss
        loss = criterion(output, target)
        # update running validation loss 
        valid_loss += loss.item()*data.size(0)
        
    # print training/validation statistics 
    # calculate average loss over an epoch
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
    
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, 
        train_loss,
        valid_loss
        ))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        torch.save(model, 'model_full.pt')
        valid_loss_min = valid_loss

Epoch: 1 	Training Loss: 1.135720 	Validation Loss: 0.429306
Validation loss decreased (inf --> 0.429306).  Saving model ...
Epoch: 2 	Training Loss: 0.445268 	Validation Loss: 0.313916
Validation loss decreased (0.429306 --> 0.313916).  Saving model ...
Epoch: 3 	Training Loss: 0.348292 	Validation Loss: 0.259261
Validation loss decreased (0.313916 --> 0.259261).  Saving model ...
Epoch: 4 	Training Loss: 0.290256 	Validation Loss: 0.221010
Validation loss decreased (0.259261 --> 0.221010).  Saving model ...
Epoch: 5 	Training Loss: 0.255953 	Validation Loss: 0.193248
Validation loss decreased (0.221010 --> 0.193248).  Saving model ...
Epoch: 6 	Training Loss: 0.224580 	Validation Loss: 0.173427
Validation loss decreased (0.193248 --> 0.173427).  Saving model ...
Epoch: 7 	Training Loss: 0.201901 	Validation Loss: 0.157909
Validation loss decreased (0.173427 --> 0.157909).  Saving model ...
Epoch: 8 	Training Loss: 0.183551 	Validation Loss: 0.144871
Validation loss decreased (0.15790

In [38]:
model.load_state_dict(torch.load('model.pt'))

In [39]:
# initialize lists to monitor test loss and accuracy
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval() # prep model for evaluation

for data, target in test_loader:
    # forward pass: compute predicted outputs by passing inputs to the model
    data=data.view(data.shape[0], -1)
    output = model(data)
    # calculate the loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)
    # compare predictions to true label
    correct = np.squeeze(pred.eq(target.data.view_as(pred)))
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# calculate and print avg test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            str(i), 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.064548

Test Accuracy of     0: 99% (971/980)
Test Accuracy of     1: 99% (1124/1135)
Test Accuracy of     2: 97% (1010/1032)
Test Accuracy of     3: 98% (994/1010)
Test Accuracy of     4: 98% (965/982)
Test Accuracy of     5: 97% (871/892)
Test Accuracy of     6: 97% (938/958)
Test Accuracy of     7: 97% (1001/1028)
Test Accuracy of     8: 97% (954/974)
Test Accuracy of     9: 97% (985/1009)

Test Accuracy (Overall): 98% (9813/10000)


In [43]:
from PIL import Image as l
o=l.open('mod.png')

#img.shape
from torchvision import transforms
m1=transforms.Grayscale().__call__(o)
m=transforms.ToTensor().__call__(m1)

#m=torch.from_numpy(img)
print(m.shape)
m=m.view(1, -1)
print(m.shape)
#x=torch.load('model.pt')

data, target=next(iter(test_loader))
# forward pass: compute predicted outputs by passing inputs to the model
dat=data[0].view(1, -1)

#print(x)
with torch.no_grad():
    output = model(dat)
    _, pred = torch.max(output, 1)

print(pred,target[0])

torch.Size([1, 28, 28])
torch.Size([1, 784])
tensor([7]) tensor(7)
